In [2]:
import torch

In [3]:
from transformers import pipeline

generator = pipeline('text-generation', model="facebook/opt-2.7b")
generator("What are we having for dinner?")

pytorch_model.bin:   0%|          | 0.00/5.30G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.30G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Device set to use mps:0


In [ ]:
import finnhub
finnhub_client = finnhub.Client(api_key="")

print(finnhub_client.general_news('general', min_id=0))